# Multi-GPU Training Ejemplo

Entrena una red neuronal convolucional en múltiples GPUs con TensorFlow.

Este ejemplo utiliza capas de TensorFlow, ver 'convolutional_network_raw' ejemplo
para una implementación de TensorFlow con variables.

## Formación con múltiples tarjetas GPU

En este ejemplo, estamos utilizando el paralelismo de datos para dividir el entrenamiento en varias GPUs. Cada GPU tiene una réplica completa del modelo de red neuronal, y los pesos (es decir, las variables) se actualizan de forma sincrónica a la espera de que cada GPU procese su lote de datos.

En primer lugar, cada GPU procesa un lote distinto de datos y calcula los degradados correspondientes; a continuación, todos los degradados se acumulan en la CPU y se promedian. Los pesos de los modelos se actualizan finalmente con los gradientes promediados, y los nuevos pesos de los modelos se devuelven a cada GPU para repetir el proceso de formación.

<img src="https://www.tensorflow.org/images/Parallelism.png" alt="Parallelism" style="width: 400px;"/>

## MNIST Descripción general del conjunto de datos

Este ejemplo utiliza dígitos manuscritos MNIST. El conjunto de datos contiene 60.000 ejemplos de formación y 10.000 ejemplos de pruebas. Los dígitos han sido normalizados y centrados en una imagen de tamaño fijo (28x28 píxeles) con valores de 0 a 1. Para simplificar, cada imagen ha sido aplanada y convertida a una matriz numérica 1-D de 784 características (28*28).

![MNIST Dataset](http://neuralnetworksanddeeplearning.com/images/mnist_100_digits.png)

Más información: http://yann.lecun.com/exdb/mnist/

In [1]:
from __future__ import print_function

import numpy as np
import tensorflow as tf
import time

# Importar datos MNIST
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("/tmp/data/", one_hot=True)

# Parametros
num_gpus = 2
num_steps = 200
learning_rate = 0.001
batch_size = 1024
display_step = 10

# Network Parametros
num_input = 784 # Entrada de datos MNIST (forma de la imagen: 28*28)
num_classes = 10 # Total de clases MNIST (0-9 dígitos)
dropout = 0.75 # Abandono, probabilidad de mantener las unidades

Extracting /tmp/data/train-images-idx3-ubyte.gz
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz


In [2]:
# Construir una convolutional neural network
def conv_net(x, n_classes, dropout, reuse, is_training):
    # Definir un ámbito de reutilización de las variables
    with tf.variable_scope('ConvNet', reuse=reuse):
        # La entrada de datos MNIST es un vector 1-D de 784 características (28*28 píxeles)
        # Modificar para que coincida con el formato de la imagen[Altura x Anchura x Canal]
        # La entrada del tensor se convierte en 4-D:[Tamaño del lote, altura, ancho, canal]
        x = tf.reshape(x, shape=[-1, 28, 28, 1])

        # Capa de convolución con 64 filtros y un tamaño de núcleo de 5
        x = tf.layers.conv2d(x, 64, 5, activation=tf.nn.relu)
        # Agrupación máxima (muestreo descendente) con zancadas de 2 y tamaño de kernel de 2
        x = tf.layers.max_pooling2d(x, 2, 2)

        # Capa de convolución con 256 filtros y un tamaño de núcleo de 5
        x = tf.layers.conv2d(x, 256, 3, activation=tf.nn.relu)
        # Capa de convolución con 512 filtros y un tamaño de núcleo de 5
        x = tf.layers.conv2d(x, 512, 3, activation=tf.nn.relu)
        # Agrupación máxima (muestreo descendente) con zancadas de 2 y tamaño de kernel de 2
        x = tf.layers.max_pooling2d(x, 2, 2)

        # Aplanar los datos a un vector 1-D para la capa completamente conectada.
        x = tf.contrib.layers.flatten(x)

        # Capa completamente conectada (en la carpeta contrib por ahora)
        x = tf.layers.dense(x, 2048)
        # Aplicar Abandono (si is_training es Falso, no se aplica el abandono)
        x = tf.layers.dropout(x, rate=dropout, training=is_training)

        # Capa completamente conectada (en la carpeta contrib por ahora)
        x = tf.layers.dense(x, 1024)
        # Aplicar Abandono (si is_training es Falso, no se aplica el abandono)
        x = tf.layers.dropout(x, rate=dropout, training=is_training)

        # Capa de salida, predicción de clases
        out = tf.layers.dense(x, n_classes)
        # Porque la pérdida de'softmax_cross_entropy_with_logits' ya se aplica
        # softmax, sólo aplicamos softmax a la red de pruebas
        out = tf.nn.softmax(out) if not is_training else out

    return out

In [3]:
# Construya la función para promediar los gradientes
def average_gradients(tower_grads):
    average_grads = []
    for grad_and_vars in zip(*tower_grads):
        # Tenga en cuenta que cada grad_y_vars tiene el siguiente aspecto:
        # ((grad0_gpu0, var0_gpu0), .... (grad0_gpuN, var0_gpuN))
        grads = []
        for g, _ in grad_and_vars:
            # Añada la dimensión 0 a los gradientes para representar la torre.
            expanded_g = tf.expand_dims(g, 0)

            # Añada una dimensión de "torre" que promediamos a continuación.
            grads.append(expanded_g)

        # Promedio sobre la dimensión 'torre'.
        grad = tf.concat(grads, 0)
        grad = tf.reduce_mean(grad, 0)

        # Tenga en cuenta que las Variables son redundantes porque son compartidas
        # a través de las torres. Así que... simplemente devolveremos el puntero de la primera torre a
        # la Variable.
        v = grad_and_vars[0][1]
        grad_and_var = (grad, v)
        average_grads.append(grad_and_var)
    return average_grads

In [4]:
# Por defecto, todas las variables se colocarán en '/gpu:0''.
# Así que necesitamos una función de dispositivo personalizada, para asignar todas las variables a'/cpu:0'.
# Nota: Si se utilizan GPUs, '/gpu:0' puede ser una opción más rápida.
PS_OPS = ['Variable', 'VariableV2', 'AutoReloadVariable']

def assign_to_device(device, ps_device='/cpu:0'):
    def _assign(op):
        node_def = op if isinstance(op, tf.NodeDef) else op.node_def
        if node_def.op in PS_OPS:
            return "/" + ps_device
        else:
            return device

    return _assign

In [5]:
# Poner todos los ops en la CPU por defecto
with tf.device('/cpu:0'):
    tower_grads = []
    reuse_vars = False

    # tf Entrada de gráficos
    X = tf.placeholder(tf.float32, [None, num_input])
    Y = tf.placeholder(tf.float32, [None, num_classes])

    # Realice un bucle en todas las GPUs y construya su propio gráfico de cálculo.
    for i in range(num_gpus):
        with tf.device(assign_to_device('/gpu:{}'.format(i), ps_device='/cpu:0')):

            # Divide los datos entre las GPUs
            _x = X[i * batch_size: (i+1) * batch_size]
            _y = Y[i * batch_size: (i+1) * batch_size]

            # Debido a que los Abandonados tienen un comportamiento diferente en el entrenamiento y en el tiempo de predicción, nosotros
            # necesitan crear 2 gráficos de cálculo distintos que compartan los mismos pesos.

            # Cree un gráfico para el entrenamiento
            logits_train = conv_net(_x, num_classes, dropout,
                                    reuse=reuse_vars, is_training=True)
            # Cree otro gráfico para probar que reutilice los mismos pesos
            logits_test = conv_net(_x, num_classes, dropout,
                                   reuse=True, is_training=False)

            # Definir la pérdida y el optimizador (con los registros del tren, para que la pérdida de datos tenga efecto)
            loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(
                logits=logits_train, labels=_y))
            optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
            grads = optimizer.compute_gradients(loss_op)

            # Sólo la primera precisión de cálculo de la GPU
            if i == 0:
                # Evaluar el modelo (con registros de pruebas, para deshabilitar la deserción escolar)
                correct_pred = tf.equal(tf.argmax(logits_test, 1), tf.argmax(_y, 1))
                accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

            reuse_vars = True
            tower_grads.append(grads)

    tower_grads = average_gradients(tower_grads)
    train_op = optimizer.apply_gradients(tower_grads)

    # Inicialización de las variables
    init = tf.global_variables_initializer()

    # Iniciar el gráfico
    with tf.Session() as sess:
        sess.run(init)
        step = 1
        # Mantenga el entrenamiento hasta alcanzar las iteraciones máximas
        for step in range(1, num_steps + 1):
            # Obtén un lote por cada GPU
            batch_x, batch_y = mnist.train.next_batch(batch_size * num_gpus)
            # Optimización de la ejecución (backprop)
            ts = time.time()
            sess.run(train_op, feed_dict={X: batch_x, Y: batch_y})
            te = time.time() - ts
            if step % display_step == 0 or step == 1:
                # Calcular la pérdida y la precisión de los lotes
                loss, acc = sess.run([loss_op, accuracy], feed_dict={X: batch_x,
                                                                     Y: batch_y})
                print("Step " + str(step) + ": Minibatch Loss= " + \
                      "{:.4f}".format(loss) + ", Training Accuracy= " + \
                      "{:.3f}".format(acc) + ", %i Examples/sec" % int(len(batch_x)/te))
            step += 1
        print("Optimization Finished!")

        # Calcule la precisión para imágenes de prueba de 1000 mnist
        print("Testing Accuracy:", \
            np.mean([sess.run(accuracy, feed_dict={X: mnist.test.images[i:i+batch_size],
            Y: mnist.test.labels[i:i+batch_size]}) for i in range(0, len(mnist.test.images), batch_size)]))

Step 1: Minibatch Loss= 2.4077, Training Accuracy= 0.123, 682 Examples/sec
Step 10: Minibatch Loss= 1.0067, Training Accuracy= 0.765, 6528 Examples/sec
Step 20: Minibatch Loss= 0.2442, Training Accuracy= 0.945, 6803 Examples/sec
Step 30: Minibatch Loss= 0.2013, Training Accuracy= 0.951, 6741 Examples/sec
Step 40: Minibatch Loss= 0.1445, Training Accuracy= 0.962, 6700 Examples/sec
Step 50: Minibatch Loss= 0.0940, Training Accuracy= 0.971, 6746 Examples/sec
Step 60: Minibatch Loss= 0.0792, Training Accuracy= 0.977, 6627 Examples/sec
Step 70: Minibatch Loss= 0.0593, Training Accuracy= 0.979, 6749 Examples/sec
Step 80: Minibatch Loss= 0.0799, Training Accuracy= 0.984, 6368 Examples/sec
Step 90: Minibatch Loss= 0.0614, Training Accuracy= 0.988, 6762 Examples/sec
Step 100: Minibatch Loss= 0.0716, Training Accuracy= 0.983, 6338 Examples/sec
Step 110: Minibatch Loss= 0.0531, Training Accuracy= 0.986, 6504 Examples/sec
Step 120: Minibatch Loss= 0.0425, Training Accuracy= 0.990, 6721 Examples/se